In [5]:
import pandas as pd
import numpy as np
import random
import sys
import networkx as nx
!{sys.executable} -m pip install "networkx[default]"
cdi = pd.read_csv('/Users/abbyhultquist/Desktop/First Year Project/CDI_cleaned 2.csv')

In [6]:
word_col = cdi.columns[19:] ; # Extracting the columns from 'baa baa' onwards
similarity = pd.read_csv('/Users/abbyhultquist/Desktop/First Year Project/nouns.csv', index_col=0)

In [29]:
import numpy as np

threshold = 0.8
idx = similarity.index
mat = similarity.values

pairs_weighted = [
    (idx[i], idx[j], float(mat[i, j]))
    for i, j in zip(*np.triu_indices_from(mat, k=1))
    if mat[i, j] > threshold
]

similarity_touples_unweighted = [
     (idx[i], idx[j])
    for i, j in zip(*np.triu_indices_from(mat, k=1))
    if mat[i, j] > threshold
]
# optional: sort by similarity, where x[2] is the second column (similarity)
pairs = sorted(pairs, key=lambda x: x[2], reverse=True)


In [ ]:
G = nx.complete_graph(5)
preds = nx.preferential_attachment(G, [(0, 1), (2, 3)])
for u, v, p in preds:
    print(f"({u}, {v}) -> {p}")



(0, 1) -> 16
(2, 3) -> 16


In [34]:
print(nx.complete_graph(5))

Graph with 5 nodes and 10 edges


In [8]:
words = sorted(set(word_col) & set(similarity.columns))
#sorted(set(list of words) & set(list of words)) intersects the two lists 

similarity = similarity.loc[words, words] #selecting words from columns and rows that are in words list

In [9]:
def predict_next_words(known_words, similarity_matrix, top_n=10):
    unknown_words = list(set(similarity_matrix.index) - set(known_words))
    
    scores = {}
    
    for w in unknown_words:
        sims = similarity_matrix.loc[w, known_words] #sims is a row of similarities for the word w and the known words
        scores[w] = sims.mean() #averages the similarity between w and all known words

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_n]

In [10]:
def random_growth(known_words, similarity_matrix, top_n=10):
    
    unknown_words = list(set(similarity_matrix.index) - set(known_words))
    return random.sample(unknown_words, min(top_n, len(unknown_words))) # if unknown words are less than top_n, return all unknown words

NameError: name 'preferential_attachment' is not defined

In [ ]:
child_id = "4139"
child_data = cdi[cdi["child_id"] == child_id].sort_values("session_num")


In [ ]:
t = 1
child_data = cdi[cdi["child_id"] == child_id].sort_values("session_num")
session_t = child_data[child_data["session_num"] == t]
session_t1 = child_data[child_data["session_num"] == t+5]

known_t  = [w for w in words if session_t[w].iat[0]  == 1]
known_t1 = [w for w in words if session_t1[w].iat[0] == 1]
# enforce monotonicity (no forgetting)
known_t1 = list(set(known_t1) | set(known_t))
print(session_t1)


  child_id study id   study gender        age    birthday session_date  \
5     4139   LTP102  Long-1      F  22.189349  2008/10/17   2010/08/23   

   session_num  total num sessions  words_spoken  ...  was  were  will  would  \
5            6                  12           288  ...    0     0     0      0   

  and  because but  if  so  then  
5   0        0   0   0   0     0  

[1 rows x 699 columns]


In [ ]:
actual_new = list(set(known_t1) - set(known_t))
#actual_new = set(known_t) - set(known_t1)

num_new = len(actual_new)
predicted = predict_next_words(known_t, similarity, top_n=num_new)
predicted_words = [w for w, score in predicted]

random_prediction = random_growth(known_t, similarity, top_n=num_new)


In [ ]:
# Calculate the accuracy of the predictions
accuracy_pref = len(set(actual_new) & set(predicted_words)) / len(actual_new)
accuracy_rand = len(set(actual_new) & set(random_prediction)) / len(actual_new)
print(accuracy_pref, "accuracy of preferential attachment prediction")
print(accuracy_rand, "accuracy of random prediction")

0.4074074074074074 accuracy of preferential attachment prediction
0.37962962962962965 accuracy of random prediction
